In [42]:
import torch
import wandb
import argparse
import os
import sys
import numpy as np

from tqdm import tqdm
from torch import nn
import numpy as np
from torch.utils.data import DataLoader
from sktime.regression.base import BaseRegressor
from sktime.regression.dummy import DummyRegressor
from sktime.regression.kernel_based import RocketRegressor
from sktime.utils import mlflow_sktime

from util.utils import set_all_seeds
from data.dataloader import Platoon

import warnings
from sklearn.exceptions import DataConversionWarning
# Ignore specific warning
warnings.filterwarnings(action='ignore', category=UserWarning, module='sktime.base._base_panel')



In [43]:
model_name = "dummy_regressor"
project_name = "Version 0.1"
seed = 42
num_epochs = 2
batch_size = 10
window_size = 10
verbose = False
only_iri = False

In [44]:
data_path = 'D:/sc_ml_and_road_conditions/CrackDetect/data/processed'
trainset = Platoon(data_type='train', data_path=data_path, window_size=window_size, only_iri=only_iri)
train_loader = DataLoader(trainset, batch_size=None, shuffle=True, num_workers=1)
valset = Platoon(data_type='val', data_path=data_path, window_size=window_size, only_iri=only_iri)
val_loader = DataLoader(valset, batch_size=None, shuffle=False, num_workers=1)

## Testing different tsai models

In [45]:
from tsai.basics import *
import sktime
import sklearn
my_setup(sktime, sklearn)
from tsai.models.MINIROCKET import *
from sklearn.metrics import mean_squared_error, make_scorer

def create_batches(data, targets, batch_size):
    num_batches = len(data) // batch_size

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        yield np.expand_dims(data[start_idx:end_idx], 1).astype('float32'), targets[start_idx:end_idx].astype('float32')

    """
    NOTE - this is a generator, so the last batch will be smaller, but for certain models 
    we have to have fixed batch size so we can't just yield the last batch
    """    
    # if len(data) % batch_size != 0:
    #     start_idx = num_batches * batch_size
    #     yield np.expand_dims(data[start_idx:end_idx], 1).astype('float32'), targets[start_idx:].astype('float32')


os              : Windows-10-10.0.19045-SP0
python          : 3.10.11
tsai            : 0.3.9
fastai          : 2.7.14
fastcore        : 1.5.29
sktime          : 0.27.0
sklearn         : 1.4.1.post1
torch           : 2.2.1+cpu
device          : cpu
cpu cores       : 6
threads per cpu : 2
RAM             : 15.93 GB
GPU memory      : [8.0] GB


In [46]:

rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
model = MiniRocketRegressor(scoring=rmse_scorer)

total_loss = []
for epoch in range(num_epochs):
    train_iterator = tqdm(train_loader, unit="batch", position=0, leave=False)
    train_loss = []
    train_iterator.set_description(f"Epoch {epoch+1}/{num_epochs}")
    for data_segment, target_segment in train_iterator:
        for data, target in create_batches(data_segment, target_segment, batch_size=batch_size):
            model.fit(data, target)
            output = model.predict(data)
            loss = mean_squared_error(target, output)
            train_loss.append(loss)
            train_iterator.set_postfix_str(f"train_loss: {loss:.6f}")
    total_loss.append(np.mean(train_loss))
    # iterator.set_postfix_str(f"train_loss: {total_loss[-1]:.6f}")

In [47]:
val_iterator = tqdm(val_loader, unit="batch", position=0, leave=False)
RMSs = []
for data_segment, target_segment in val_iterator:
    for data, target in create_batches(data_segment, target_segment, batch_size=batch_size):
        y_pred = model.predict(data)
        rmse = mean_squared_error(target, y_pred, squared=False)
        RMSs.append(rmse)
        val_iterator.set_postfix_str(f"val_loss: {rmse:.6f}")

print(f"Mean RMSE: {np.mean(RMSs):.6f}")

Mean RMSE: 0.344274
